This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [23]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [24]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "C:/Users/heral/Downloads/Train1"  
test_dir = "C:/Users/heral/Downloads/Test1"    

In [25]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
# ResNet50 expects at least 197x197
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [26]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
# Use ResNet preprocessing function for transfer learning
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [27]:
# Transfer learning with ResNet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Build base model with ImageNet weights (no top)
base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base.trainable = False  # freeze base layers for initial training

# Add a simple classification head
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base.input, outputs=output)

# Compile with a small learning rate for the head
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [28]:
# TRAIN ResNet50 classifier head and fine-tune
from tensorflow.keras import callbacks as cb
es = cb.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rlp = cb.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the top classifier head first
import math
# cap steps per epoch to 120 to shorten each epoch (useful for large datasets or limited time)
max_steps = 120
steps_per_epoch = min(max_steps, math.ceil(train_generator.n / train_generator.batch_size))
val_steps = min(30, math.ceil(val_generator.n / val_generator.batch_size))
# change head-training epochs to 10 as requested
history = model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[es, rlp], steps_per_epoch=steps_per_epoch, validation_steps=val_steps)

# Unfreeze last convolutional block for fine-tuning
for layer in base.layers[-30:]:
    layer.trainable = True

# Re-compile with lower LR for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history_ft = model.fit(train_generator, validation_data=val_generator, epochs=5, callbacks=[es, rlp])

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# Save final fine-tuned model
model.save('exercise_6_resnet_amolong.h5')
print('Saved exercise_6_resnet_amolong.h5')

Epoch 1/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.5773 - loss: 0.7808 - val_accuracy: 0.8427 - val_loss: 0.4266 - learning_rate: 1.0000e-04
Epoch 2/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.5773 - loss: 0.7808 - val_accuracy: 0.8427 - val_loss: 0.4266 - learning_rate: 1.0000e-04
Epoch 2/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.8198 - loss: 0.4040 - val_accuracy: 0.9354 - val_loss: 0.2503 - learning_rate: 1.0000e-04
Epoch 3/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.8198 - loss: 0.4040 - val_accuracy: 0.9354 - val_loss: 0.2503 - learning_rate: 1.0000e-04
Epoch 3/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.9005 - loss: 0.2701 - val_accuracy: 0.9604 - val_loss: 0.1677 - learning_rate: 1.0000e-04
Epoch 4/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.9005 - loss: 0.2701 - val_accuracy: 0.9604 - val_loss: 0.1677 - learning_rate: 1.0000e-04
Epoch 4/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - acc

c:\Users\heral\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


120/120 ━━━━━━━━━━━━━━━━━━━━ 48s 396ms/step - accuracy: 0.9250 - loss: 0.1991 - val_accuracy: 0.9615 - val_loss: 0.1317 - learning_rate: 1.0000e-04
Epoch 6/10
Epoch 6/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.9445 - loss: 0.1653 - val_accuracy: 0.9635 - val_loss: 0.1196 - learning_rate: 1.0000e-04
Epoch 7/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.9445 - loss: 0.1653 - val_accuracy: 0.9635 - val_loss: 0.1196 - learning_rate: 1.0000e-04
Epoch 7/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.9503 - loss: 0.1466 - val_accuracy: 0.9719 - val_loss: 0.1009 - learning_rate: 1.0000e-04
Epoch 8/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.9503 - loss: 0.1466 - val_accuracy: 0.9719 - val_loss: 0.1009 - learning_rate: 1.0000e-04
Epoch 8/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - accuracy: 0.9586 - loss: 0.1295 - val_accuracy: 0.9719 - val_loss: 0.0863 - learning_rate: 1.0000e-04
Epoch 9/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - a

Test Accuracy: 0.9898
Saved exercise_6_resnet_amolong.h5
Saved exercise_6_resnet_amolong.h5


In [29]:
model = tf.keras.models.load_model('exercise_6_resnet_amolong.h5')
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 149s 938ms/step - accuracy: 0.9898 - loss: 0.0275
Test Accuracy: 0.989799976348877
157/157 ━━━━━━━━━━━━━━━━━━━━ 149s 938ms/step - accuracy: 0.9898 - loss: 0.0275
Test Accuracy: 0.989799976348877


In [30]:
model.save('exercise_6_resnet_amolong.h5')
print('Model saved during training as exercise_6_resnet_amolong.h5')

Model saved during training as exercise_6_resnet_amolong.h5


In [35]:
# SIMPLE INFERENCE SCRIPT FOR ResNet MODEL
from tensorflow.keras.preprocessing import image
import os

def predict_image(img_path, model_path='exercise_6_resnet_amolong.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    # ResNet preprocess was applied in the generator; apply same here
    img_array = tf.keras.applications.resnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    pred = float(model.predict(img_array, verbose=0)[0,0])
    label = 'Dog' if pred >= 0.5 else 'Car'
    print(f"{os.path.basename(img_path)} -> Prediction: {label} (confidence: {pred:.3f})")
    return label, pred


In [36]:
# Example usage (change paths to actual images):
predict_image(r'C:/Users/heral/Downloads/cat1.jpg')
predict_image(r'C:/Users/heral/Downloads/cat2.jpg')
predict_image(r'C:/Users/heral/Downloads/cat3.jpg')
predict_image(r'C:/Users/heral/Downloads/cat4.jpg')
predict_image(r'C:/Users/heral/Downloads/cat5.jpg')

predict_image(r'C:/Users/heral/Downloads/dog1.jpg')
predict_image(r'C:/Users/heral/Downloads/dog2.jpg')
predict_image(r'C:/Users/heral/Downloads/dog3.jpg')
predict_image(r'C:/Users/heral/Downloads/dog4.jpg')
predict_image(r'C:/Users/heral/Downloads/dog5.jpg')

cat1.jpg -> Prediction: Car (confidence: 0.000)


cat2.jpg -> Prediction: Car (confidence: 0.000)


cat3.jpg -> Prediction: Car (confidence: 0.000)


cat4.jpg -> Prediction: Car (confidence: 0.000)


cat5.jpg -> Prediction: Car (confidence: 0.000)


...

dog2.jpg -> Prediction: Dog (confidence: 1.000)


dog3.jpg -> Prediction: Dog (confidence: 1.000)


dog4.jpg -> Prediction: Dog (confidence: 1.000)


dog5.jpg -> Prediction: Dog (confidence: 1.000)


('Dog', 0.9999865889549255)